---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )

    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.c
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    import pandas as pd
    university_town = pd.read_table('university_towns.txt',header = None)
    university_town.rename(columns = {0:'RegionName'}, inplace = True)
    state = [x for x in university_town.iloc[:,0] if 'edit' in x]
    state_index = []
    for std in state:
        state_index.append(university_town.index[university_town.iloc[:,0] == std])
    for i in range(len(state_index)):
        university_town.loc[state_index[i],'State'] = state[i]
    university_town = university_town[['State','RegionName']]
    university_town.fillna(method = 'ffill',inplace = True)
    university_town  = university_town[university_town['RegionName'] != university_town['State']]
    university_town['RegionName'] = university_town['RegionName'].apply(lambda x: x.split('(')[0].strip())
    university_town['State'] = university_town['State'].apply(lambda x: x.split('[')[0].strip())
    
    
    
    return university_town

In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',header =4)
    gdp = gdp.iloc[2:,4:7 ]
    gdp = gdp.reset_index()
    del gdp['index']
    gdp.rename(columns = {'Unnamed: 4' : 'quarter'}, inplace = True)
    gdp_before = gdp['GDP in billions of chained 2009 dollars.1'][:len(gdp)-1]
    gdp_after = gdp['GDP in billions of chained 2009 dollars.1'][1:]
    gdp_before = gdp_before.reset_index()
    gdp_after = gdp_after.reset_index()
    del gdp_before['index']
    del gdp_after['index']
    deornot = (gdp_before - gdp_after) > 0
    gdp.loc[1:,'deornot'] = deornot.values
    gdp = gdp[gdp['quarter'] > '2000']
    gdp = gdp.reset_index()
    del gdp['index']
    for i in range(len(gdp)):
        for k in range(i+2,len(gdp)):
            if (gdp['deornot'][i]==False and gdp['deornot'][i+1]==True and gdp['deornot'][i+2]==True)and (gdp['deornot'][k]==False and gdp['deornot'][k+1]==False):
                start = gdp['quarter'][i+1]
                break
        

    
    
    return start

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',header =4)
    gdp = gdp.iloc[2:,4:7 ]
    gdp = gdp.reset_index()
    del gdp['index']
    gdp.rename(columns = {'Unnamed: 4' : 'quarter'}, inplace = True)
    gdp_before = gdp['GDP in billions of chained 2009 dollars.1'][:len(gdp)-1]
    gdp_after = gdp['GDP in billions of chained 2009 dollars.1'][1:]
    gdp_before = gdp_before.reset_index()
    gdp_after = gdp_after.reset_index()
    del gdp_before['index']
    del gdp_after['index']
    deornot = (gdp_before - gdp_after) > 0
    gdp.loc[1:,'deornot'] = deornot.values
    gdp = gdp[gdp['quarter'] > '2000']
    gdp = gdp.reset_index()
    del gdp['index']
    for i in range(len(gdp)):
        for k in range(i+2,len(gdp)):
            if (gdp['deornot'][i]==False and gdp['deornot'][i+1]==True and gdp['deornot'][i+2]==True)and (gdp['deornot'][k]==False and gdp['deornot'][k+1]==False):
                end = gdp['quarter'][k+1]
                break
        

    
    
       
    return end

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',header =4)
    gdp = gdp.iloc[2:,4:7 ]
    gdp = gdp.reset_index()
    del gdp['index']
    gdp.rename(columns = {'Unnamed: 4' : 'quarter'}, inplace = True)
    gdp_before = gdp['GDP in billions of chained 2009 dollars.1'][:len(gdp)-1]
    gdp_after = gdp['GDP in billions of chained 2009 dollars.1'][1:]
    gdp_before = gdp_before.reset_index()
    gdp_after = gdp_after.reset_index()
    del gdp_before['index']
    del gdp_after['index']
    deornot = (gdp_before - gdp_after) > 0
    gdp.loc[1:,'deornot'] = deornot.values
    gdp = gdp[gdp['quarter'] > '2000']
    gdp = gdp.reset_index()
    del gdp['index']
    for i in range(len(gdp)):
        for k in range(i+2,len(gdp)):
            if (gdp['deornot'][i]==False and gdp['deornot'][i+1]==True and gdp['deornot'][i+2]==True)and (gdp['deornot'][k]==False and gdp['deornot'][k+1]==False):
                startidx = i+1
                endidx = k+1
                break
    idxmin = gdp['GDP in billions of chained 2009 dollars.1'][startidx:endidx+1].idxmin()
    recesstion_bottom = gdp['quarter'][idxmin]

    
    
    return recesstion_bottom

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    house = pd.read_csv('City_Zhvi_AllHomes.csv')
    keepcolumns = []
    keepcolumns.extend(house.columns[house.columns >= '2000-01'][[1,2]])
    keepcolumns.extend(house.columns[house.columns >= '2000-01'][6:])
    house = house[keepcolumns]
    house['State'].replace(states,inplace = True)
    house.set_index(['State','RegionName'],inplace=True)
    house.columns = pd.to_datetime(house.columns)
    house = house.resample('Q', axis =1).mean()
    for i in range(len(house.columns)):
        if str(house.columns[i]).split('-')[1] == '03':
            house.rename(columns = {house.columns[i] : str(house.columns[i]).split('-')[0] + 'q1'},inplace = True)
        elif str(house.columns[i]).split('-')[1] == '06':
            house.rename(columns = {house.columns[i] : str(house.columns[i]).split('-')[0] + 'q2'},inplace = True)
        elif str(house.columns[i]).split('-')[1] == '09':
            house.rename(columns = {house.columns[i] : str(house.columns[i]).split('-')[0] + 'q3'},inplace = True)
        elif str(house.columns[i]).split('-')[1] == '12':
            house.rename(columns = {house.columns[i] : str(house.columns[i]).split('-')[0] + 'q4'},inplace = True)
    
    return house

In [66]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    house = convert_housing_data_to_quarters()
    university_town = get_list_of_university_towns()
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    university_list = university_town.to_records(index=False).tolist()
    house_university = house.loc[university_list]
    house_not_university = house[list(house.reset_index()['RegionName'].apply(lambda x : not x in university_list))]
    house_university_recession = house_university.loc[:,[recession_start,recession_bottom]]
    house_not_university_recession = house_not_university.loc[:,[recession_start,recession_bottom]]
    house_university_recession['diff'] = house_university_recession[recession_start].div(house_university_recession[recession_bottom])
    house_not_university_recession['diff'] = house_not_university_recession[recession_start].div(house_not_university_recession[recession_bottom])
    t_test = ttest_ind(house_not_university_recession['diff'],house_university_recession['diff'],nan_policy='omit')
    p = t_test[1]
    
    if p <0.01:
        different = True
    else:
        different = False
    
    if house_not_university_recession['diff'].sum() < house_university_recession['diff'].sum():
        better = "non-university town"
    else:
        better = "university town"
    return different, p, better